<div class="alert alert-info" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid " >
    
<b> Ссылки на дашборд и презентацию: </b>   
- Дашборд: https://public.tableau.com/app/profile/yana.shinkaryuk/viz/_16915720611960/sheet2?publish=yes    
- Презентация: https://drive.google.com/file/d/18uKahZUlsFi5iol0vRfyWvkOq4m9c2id/view?usp=sharing    
    
</div>

# Анализ поведения пользователей в мобильном приложении   
**Описание проекта:**  

Вы находитесь на позиции джуниор-аналитика в составе продуктовой команды сервиса «Ненужные вещи». В нем пользователи продают свои ненужные вещи, размещая их на доске объявлений. Заказчик - продакт менеджер, который занимается вовлечением пользователей в приложение. На основе нашего исследования заказчик будет принимать решение о сегментации пользователей на группы, которая позволит в последствии более эффективно влиять на их вовлечение.   

**Описание данных:**  
Датасет содержит данные о событиях (месячная выгрузка логов новых пользователей), совершенных в мобильном приложении "Ненужные вещи", после 7 октября 2019 года.  

Колонки в *mobile_sources.csv:*  
- userId — идентификатор пользователя  
- source — источник, с которого пользователь установил приложение  

Колонки в *mobile_dataset.csv:*  
- event.time — время совершения действия  
- user.id — идентификатор пользователя  
- event.name — действие пользователя  

Виды действий:  
- advert_open — открыл карточки объявления  
- photos_show — просмотрел фотографий в объявлении  
- tips_show — увидел рекомендованные объявления  
- tips_click — кликнул по рекомендованному объявлению  
- contacts_show и show_contacts — посмотрел номер телефона  
- contacts_call — позвонил по номеру из объявления  
- map — открыл карту объявлений   
- search_1 — search_7 — разные действия, связанные с поиском по сайту  
- favorites_add — добавил объявление в избранное  


**Цели исследования:**  
- Управление вовлеченностью клиентов (адаптация приложения по целевой и смежной аудитории) на основе данных о поведении пользователей  
- Выделение на основе поведения пользователей гипотезы о том, как можно было бы улучшить приложение с точки зрения пользовательского опыта  


**Ход исследования:**  

Проект будет состоять из пяти этапов:  
1. Обзор данных:  
- получение общей информации об исходных данных
- формирование первого представления о полученных данных  
- выявление проблем в данных, которые необходимо исправить на этапе предобработки  

2. Предобработка данных:  
- исправление нарушений в стиле заголовков столбцов (при необходимости) 
- проверка данных на наличие дубликатов и их удаление (при необходимости)
- проверка данных на наличие пропущенных значений и их заполнение (при необходимости)  
- проверка соответствия указанных типов данных и их изменение (при необходимости) 

3. Исследовательский анализ данных:    

3.1 Анализ связи между целевым событием (просмотр контактов) и другими действиями пользователей:  
- выделение пользовательских сессий  
- выявление в разрезе сессий сценариев поведения пользователей, которые приводят к просмотру контактов  
- построение воронок по основным сценариям поведения в разрезе уникальных пользователей  
- сравнение время между событиями map -> contacts_show (открытие карты + просмотр контактов) и search -> contacts_show (поиск + просмотр контактов) в рамках одной сессии  


3.2 Выявление наиболее частых действий пользователей, которые просматривают контакты:  
- рассчет относительной частоты событий в разрезе групп пользователей с действием contacts_show и без него (тех, кто смотрит контакты и тех, кто не смотрит контакты)


4. Проверка гипотез:  
- проверка гипотезы о равенстве конверсий в просмотры контактов у двух групп пользователей: совершивших действие tips_show и совершивших действие tips_click (тех, кто увидел рекомендованные объявления и тех, кто кликнул по рекомендованному объявлению)  
- проверка гипотезы о равенстве длительности сессий у двух групп пользователей: совершивших целевое действие contacts_show и тех, кто не совершал целевое действие  

5. Выводы и рекомендации по результатам исследования

## Обзор данных

In [ ]:
# импорт библиотек
import pandas as pd
import scipy.stats as stats
import numpy as np
import seaborn as sns
import plotly.express as px
from matplotlib import pyplot as plt
from plotly import graph_objects as go
import io
import requests
from datetime import datetime

from tqdm import tqdm

In [ ]:
# прочитаем csv-файлы и сохраним данные в переменные
try:
    events = pd.read_csv('./mobile_dataset.csv')
    sources = pd.read_csv('./mobile_sourсes.csv')
except:
    events = pd.read_csv('https://code.s3.yandex.net/datasets/mobile_dataset.csv')
    sources = pd.read_csv('https://code.s3.yandex.net/datasets/mobile_sources.csv')

In [ ]:
# выведем первые 5 строк датафрейма events
events.head()

In [ ]:
# выведем основную информацию о датафрейме events с помощью метода info():
events.info()

Датафрейм *events* содержит информацию о действиях пользователей в виде: дата и время - действие - id.  
Пропущенных значений не обнаружено, на этапе предобработки следует привести названия столбцов к единому стилю и исправить формат в столбце с датой и временем на соответствующий.

In [ ]:
# выведем первые 5 строк датафрейма sources
sources.head()

In [ ]:
# выведем основную информацию о датафрейме sources с помощью метода info():
sources.info()

Датафрейм *sources* содержит информацию об источниках привлечения пользователей в виде: id - источник.    
Пропущенных значений также не обнаружено, на этапе предобработки следует исправить название столбца userId.

## Предобработка данных  

### Работа со стилем заголовков

In [ ]:
# приведем названия столбцов датафрейма events к единому стилю
events.rename(
    {'event.time': 'event_time', 'event.name': 'event_name', 'user.id': 'user_id'}, inplace=True, axis=1)

events.columns

In [ ]:
# приведем название столбца userId в датафрейме к единому стилю
sources.rename({'userId':'user_id'}, inplace=True, axis=1)

sources.columns

### Работа с типами данных 

In [ ]:
# посчитаем и сохраним время каждого события в секундах
events['timestamp'] = events['event_time'].apply(
    (lambda x: int(datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f').timestamp())))

In [ ]:
# изменим тип данных в столбце с датой и округлим время до 1 секунды
events['event_time'] = pd.to_datetime(events['event_time']).dt.round('1S')

events.dtypes

### Добавление данных 

Выделим из столбца с полной датой и временем отдельно дату и сохраним в отдельный столбец event_date:

In [ ]:
# дата
events['event_date'] = events['event_time'].dt.date

Данные в остальных столбцах не требуют изменений, поэтому переходим к следующему разделу предобработки.

### Работа с дубликатами

#### Проверим данные на наличие явных дубликатов:

In [ ]:
print('Количество явных дубликатов в датафрейме events:', events.duplicated().sum())

In [ ]:
print('Количество явных дубликатов в датафрейме sourсes:', sources.duplicated().sum())

Явных дубликатов в исходных данных не обнаружено.

#### Перейдем к поиску неявных дубликатов:

In [ ]:
# проверим, какие источники представлены в датафрейме sourсes
sources['source'].value_counts()

В данных представлено всего 3 источника привлечения пользователей - yandex, google и объединенный для всех остальных источников - other. Неявных дубликатов среди источников не обнаружено.

In [ ]:
# проверим, какие действия представлены в датафрейме events
events['event_name'].value_counts()

Обнаружены несколько схожих пар действий:
- contacts_show и show_contacts — просмотр номера телефона  
- search_1 - search_7 - поиск в приложении    

Предлагаю объединить эти схожие пары действий для удобства дальнейшего анализа:

In [ ]:
# изменим действие show_contacts на contacts_show
events['event_name'].replace('show_contacts', 'contacts_show', inplace=True)

# изменим действия search_1 - search_7 на search
events['event_name'] = events['event_name'].str.replace('search_+\d', 'search', regex=True)
                              
                            
# проверим изменения
events['event_name'].value_counts()

### Объединение датафреймов

Оба датафрейма содержат информацию о пользователях (user_id). Проверим их на пересечение:  

In [ ]:
# сохраним уникальных пользователей из датафрейма events
users_1 = events['user_id'].unique()

print('Количество уникальных пользователей, датафрейм events:',len(users_1))

In [ ]:
# сохраним уникальных пользователей из датафрейма sourсes
users_2 = sources['user_id'].unique()
print('Количество уникальных пользователей, датафрейм sourсes:',len(users_2))

Количество уникальных пользователей совпадает. Проверим на пересечение элементы множеств users_1 и users_2:

In [ ]:
print(f'Количество совпадающих пользователей в датафреймах: {len(set(users_1) & set(users_2))}')

ID пользователей полностью идентичны. Объединим датафреймы events и sourсes по столбцу user_id:

In [ ]:
data = events.merge(sources, on='user_id', how='left')
data.head()

## Исследовательский анализ данных

### Посчитаем, сколько колько всего действий и уникальных пользователей представлено в логах:

In [ ]:
# подсчет количества событий:
print('Количество действий:', len(data))

# подсчет количества уникальных пользователей:
print('Всего уникальных пользователей в логах:', data['user_id'].nunique())

Проверим, данными за какой период мы располагаем.  
Найдем максимальную и минимальную дату исследования, а также визуализируем распределение данных во времени:

In [ ]:
# определим период, за который представлены события в логах пользователей
print('Начало исследования:', data['event_date'].min())
print('Конец исследования:', data['event_date'].max())
print('Период исследования:', data['event_date'].max() - data['event_date'].min())

In [ ]:
# визуализируем распределение данных во времени
sns.histplot(data['event_time'], bins = 50).set(
                                    title='Распределение действий пользователей по дням',
                                    xlabel='Дата',                                   
                                    ylabel='Количество пользователей')
plt.xticks(rotation=45)
plt.tight_layout()

Логи пользователей представлены за 27 дней. Данные имеются за весь период, распределены равномерно, поэтому в дальнейшем исследовании будет использован весь период целиком.

### Посчитаем количество пользователей в разрезе источников привлечения:

In [ ]:
# посчитаем количество пользователей в зависимости от источника привлечения
users_by_source = pd.DataFrame(
    data.groupby('source')['user_id'].nunique().sort_values(ascending=False).reset_index())

# переименуем столбцы
users_by_source.columns = ['source', 'users_amount']
# посчитаем долю каждой из групп пользователей от их общего количества
users_by_source['share_of_users'] = round(users_by_source['users_amount'] / data['user_id'].nunique() * 100, 1)

users_by_source.style.background_gradient(cmap='RdYlGn', subset='share_of_users')

In [ ]:
# сгруппируем пользователей по источнику привлечения и дате
event_by_source = data.groupby(['source', 'event_date']).agg({'user_id': 'nunique'}).reset_index()

# визуализируем распределение пользователей по источникам
fig = px.bar(event_by_source.sort_values(by='user_id', ascending=False), 
             x='event_date', 
             y='user_id', 
             color='source',
             title='Распределение пользователей по источникам')   

fig.update_layout(xaxis_title='Дата',
                  yaxis_title='Количество пользователей')
             
fig.show('notebook')

Самым популярным каналом привлечения пользователей является Яндекс - привлечено 45% от общего числа уникальных пользователей, второе место занимает группа других источников (other, 28.7% пользователей). Меньше всего привлечено пользователей через поисковую систему google - всего 26.3%. Следует обратить внимание на данный канал привлечения, так как google является одной из популярных поисковых систем, и может существенно увеличить приток новых пользователей в приложение.

### Посчитаем, сколько пользователей совершали каждое из действий:

In [ ]:
# сгруппируем данные по типу действий и посчитаем количество пользователей для каждого действия
users_amount = data.groupby('event_name').agg(
           {'user_id': 'nunique'}).sort_values(
                                                by='user_id', 
                                                ascending=False).reset_index()
# переименуем столбцы
users_amount.columns = ['event_name', 'users_amount']
# посчитаем долю пользователей, совершивших каждое из действий, от общего количества пользователей
users_amount['share_of_users'] = round(users_amount['users_amount'] / data['user_id'].nunique() * 100, 1)

users_amount.style.background_gradient(cmap='RdYlGn', subset='share_of_users')

Самое частое действие в приложении - показ рекомендованных объявлений, их увидели 65.2% пользователей.   
На втором месте находится поиск - им воспользовались 38.8% пользователей. Около 34% пользователей просматривают карту и 25.5% фотографии товаров. Целевое действие - просмотр контактов, совершают всего около 23% пользователей, что говорит о довольно низкой конверсии в приложении.  

Реже всего пользователи звонят через приложение (5% пользователей), кликают по рекомендованным объявлениям (7.5% пользователей) и добавляют товары в избранное (8.2% пользователей).  

Показ рекомендованных объявлений (действие tips_show) является единственным действием, которое не инициируется пользователем, а происходит автоматически, то есть не отражает какие-либо поведенческие особенности самого пользователя.

### Посчитаем, сколько приходится событий на каждого пользователя:

In [ ]:
# сгруппируем данные по user_id пользователей и посчитаем количество событий
ev_per_user = data.groupby(['user_id'], as_index=False)\
    .agg({'event_time':'count'})\
    .rename(columns={'event_time':'event_count'})\
    .sort_values(by='event_count', ascending=False)
ev_per_user

In [ ]:
# построим распределение количества событий на пользователя
sns.histplot(ev_per_user['event_count'], bins = 50).set(
                                    title='Распределение количества событий на пользователя',
                                    xlabel='Количество событий на одного пользователя',                                   
                                    ylabel='Количество пользователей')
plt.xticks(rotation=90)
plt.tight_layout()

Как мы видим, большая часть значений находится в диапазоне до 100 событий на одного пользователя.   
Увеличим масштаб графика:

In [ ]:
sns.histplot(ev_per_user['event_count'], bins=100).set(
    title='Распределение количества событий на пользователя',
    xlabel='Количество событий на одного пользователя',                                   
    ylabel='Количество пользователей',
    xlim = [0, 100])

plt.show()

In [ ]:
# посмотрим на характерные значения сведений о количестве событий на пользователя
ev_per_user.describe()

Минимальное количество событий на пользователя - одно, максимальное - 478. 
Видно, что в распределении присутствуют аномально большие значения, так как среднее количество событий на пользователя - 17, а медианное - 9.

In [ ]:
# построим график боксплот, чтобы посмотреть на распределение без аномалий
import warnings
warnings.simplefilter('ignore')

sns.boxplot(ev_per_user['event_count'], showfliers=False).set(
    title='Распределение количества событий на одного пользователя',
    xlabel='Количество событий на одного пользователя',
    ylabel='Количество пользователей')

plt.show()

В основном, на каждого пользователя приходится от 5 до 17 событий.   

In [ ]:
# посчитаем выборочные процентили по количеству событий на пользователя (95-й и 99-й):  
np.percentile(ev_per_user['event_count'], [95, 99])

Не более 5% пользователей совершали больше 59 действий в приложении и не более 1% более 132 действий.  
Примем количество событий больше 59 - за аномальное и исключим из дальнейшего анализа:

In [ ]:
ev_per_user_filtered = ev_per_user.query('event_count <= 59')
ev_per_user_filtered.head()

In [ ]:
# посмотрим на характерные значения сведений о количестве событий на пользователя по очищенным данным
ev_per_user_filtered.describe()

По очищенным данным, среднее количество событий на пользователя теперь 12, медианное - 8.

In [ ]:
# посчитаем количество потерянных событий после фильтрации
print('Потеря событий: {:.1%}'.format(
    1 - (len(ev_per_user_filtered) / len(ev_per_user))))

Доля потерянных событий составляет 4.9%, следовательно влияние фильтрации событий на дальнейший анализ будет минимально, либо совсем отсутствовать. Применим фильтрацию к исходным данным:

In [ ]:
data = data.loc[data['user_id'].isin(ev_per_user_filtered['user_id'])]

### Определим время бездействия пользователей

In [ ]:
# остортируем события по id пользователей + дате и времени 
data.sort_values(['user_id', 'event_time'], inplace=True)

# сгруппируем события по id пользователей
t_diff = data.groupby('user_id', as_index=False).agg({'timestamp':list})

# посчитаем разницу между соседними событиями и сохраним ее в столбец diffs
t_diff['diffs'] = t_diff['timestamp'].apply(lambda x: [x[i] - x[i - 1] for i in range(1, len(x))])

# исключим сессии с одним событием
t_diff = t_diff[t_diff['timestamp'].apply(lambda x: len(x) > 1)]

t_diff.head()

In [ ]:
# сохраним в переменную all_diffs посчитанное время между событиями
all_diffs = t_diff['diffs'].explode()

# посмотрим на характерные значения времени между событиями
print('Среднее время между событиями:', round(all_diffs.mean() / 60, 2), 'минут')
print('Медианное время между событиями:', round(all_diffs.median() / 60, 2), 'минут')

Посчитаем процентили, чтобы выявить нехарактерные значения для времени между соседними событиями:

In [ ]:
np.percentile(all_diffs, [90, 95, 99]) / 60

Нехарактерными значениями времени между событиями являются:   
17.78 минут, 903.9 минуты (около 15 часов), 8697.44 минут ( ≈ 144 часов или около 6 суток).

Для дальнейшего исследования, выберем значение 90 процентиля (17.78 минут) за значение **тайм-аута** - такого интервала времени бездействия пользователя, после которого считается, что пользовательская сессия закончилась.   
Для удобства расчетов, округлим значение до **18 минут:**

### Выделим сессии пользователей по принципу тайм-аута

Cоздадим отдельный столбец session_id с номерами уникальных сессий пользователей:

In [ ]:
# осотрируем данные по значениям user_id и дате со временем события
data = data.sort_values(['user_id', 'event_time'])

# для каждого пользователя определим разницу во времени между сессиями более 11 минут
g = (data.groupby('user_id')['event_time'].diff() > pd.Timedelta('18Min')).cumsum()

# создаем счетчик cессий
data['session_id'] = data.groupby(['user_id', g], sort=False).ngroup() + 1

# проверим результат
data.head()

### Поиск сценариев

Для выявления сценариев поведения пользователей воспользуемся диаграммой Санкея. Данная диаграмма позволит визуалилировать сценарии поведения пользователей в приложении, и определить самые популярные из них. Толщина потока на диаграме будет отображать популярность того или иного сценария. Подготовим данные для построения диаграммы:

In [ ]:
# удалим повторяющиеся события в рамках сессий
table = data.drop_duplicates(subset=['session_id', 'event_name'], keep='last')

Так как действие tips_show - показ рекомендованных объявлений, генерируется автоматически самим сервисом, то есть пользователи его не инициируют самостоятельно, предлагаю его исключить перед построением диаграммы:

In [ ]:
# исключим действие tips_show
table = table.query('event_name != "tips_show"')

In [ ]:
# переименуем действия пользователей на русский язык для наглядности:
table['event_name'] = table['event_name'].replace(
    {'advert_open': 'открытие карточки',
     'photos_show': 'просмотр фотографий',
     'tips_click': 'клик по рекомендации',
     'contacts_show': 'просмотр контактов',
     'contacts_call': 'звонок',
     'map': 'открытие карты',
     'search': 'поиск',
     'favorites_add': 'добавление в избранное'}, regex=True)

Напишем функцию, которая выполнит следующие преобразования исходной таблицы:
- события каждого пользователя отсортирует по времени;
- создаст пары событий source - target;
- добавит шаг между этими событиями для построения диаграммы;
- удалит столбец event_name, так как в дальнейших преобразованиях он использоваться не будет:

In [ ]:
def add_features(df):
    
    """Функция генерации новых столбцов для исходной таблицы

    Args:
        df (pd.DataFrame): исходная таблица.
    Returns:
        pd.DataFrame: таблица с новыми признаками.
    """
    
    # сортируем по id сессии и времени
    sorted_df = df.sort_values(by=['session_id', 'event_time']).copy()
    # добавляем шаги событий
    sorted_df['step'] = sorted_df.groupby('session_id').cumcount() + 1
    
    # добавляем узлы-источники и целевые узлы
    # узлы-источники - это сами события
    sorted_df['source'] = sorted_df['event_name']
    # добавляем целевые узлы
    sorted_df['target'] = sorted_df.groupby('session_id')['source'].shift(-1)
    
    # возврат таблицы без имени событий
    return sorted_df.drop(['event_name'], axis=1)
  
# преобразуем таблицу
table = add_features(table)
table.head()

Выберем количество шагов для нашей будущей диаграммы. Чем больше шагов, тем больше графических объектов в итоге будет отображено. Для этого посмотрим на распределение уникальных событий (переходов) в разрезе сессий:

In [ ]:
# сгруппируем данные по id сессии и номеру шага
steps = table.groupby(['session_id', 'step'], as_index=False).agg({'source':list,'target':list})
steps.head()

In [ ]:
# посмотрим на распределение количества шагов
steps['step'].value_counts()

Чаще всего шагов в разрезе сессий пользователей представлено от 1 до 4. Ограничим диаграмму 4 шагами.  
Удалим все пары source-target, шаг которых превышает 4 и сохраним полученную таблицу в отдельную переменную:

In [ ]:
df_comp = table[table['step'] <= 4].copy().reset_index(drop=True)

Важным следующим шагом в подготовке данных для построения диаграммы является создание индексов для source. На каждом следующем шаге target становится source, и чтобы диаграмма коррректно генерировалась нужна правильная индексация source на каждом шаге.  

Создадим словарь, в котором ключи - это шаги, а значения - словари со списком названий source и соответствующих им индексов. На следующем шаге индексы source будут продолжать нумерацию, а не начинать с 0, при том, что имена событий могут повторяться. Затем для каждого шага объединим имена и индексы в еще один вложенный словарь. Все вложенные списки и словари потребуются в дальнейшем для генерации меток, подписей и размера каналов между source и target.  

In [ ]:
def get_source_index(df):
    
    """Функция генерации индексов source

    Args:
        df (pd.DataFrame): исходная таблица с признаками step, source, target.
    Returns:
        dict: словарь с индексами, именами и соответсвиями индексов именам source.
    """
    
    res_dict = {}
    
    count = 0
    # получаем индексы источников
    for no, step in enumerate(df['step'].unique().tolist()):
        # получаем уникальные наименования для шага
        res_dict[no+1] = {}
        res_dict[no+1]['sources'] = df[df['step'] == step]['source'].unique().tolist()
        res_dict[no+1]['sources_index'] = []
        for i in range(len(res_dict[no+1]['sources'])):
            res_dict[no+1]['sources_index'].append(count)
            count += 1
            
    # соединим списки
    for key in res_dict:
        res_dict[key]['sources_dict'] = {}
        for name, no in zip(res_dict[key]['sources'], res_dict[key]['sources_index']):
            res_dict[key]['sources_dict'][name] = no
    return res_dict
  

# создаем словарь
source_indexes = get_source_index(df_comp)

Для более наглядного представления можно разукрасить каждый source-target в разные цвета.  
Цвет зададим заранее подготовленной палитрой. Для этого создадим еще один словарь, в котором будут храниться соответствия source:color

In [ ]:
# функция для задания цветов парам source-target
def colors_for_sources(mode):
    
    """Генерация цветов rgba

    Args:
        mode (str): сгенерировать случайные цвета, если 'random', а если 'custom' - 
                    использовать заранее подготовленные
    Returns:
        dict: словарь с цветами, соответствующими каждому индексу
    """
    # словарь, в который сложим цвета в соответствии с индексом
    colors_dict = {}
    
    if mode == 'random':
        # генерим случайные цвета
        for label in df_comp['source'].unique():
            r, g, b = np.random.randint(255, size=3)            
            colors_dict[label] = f'rgba({r}, {g}, {b}, 1)'
            
    elif mode == 'custom':
        # присваиваем ранее подготовленные цвета
        colors = requests.get('https://raw.githubusercontent.com/rusantsovsv/senkey_tutorial/main/json/colors_senkey.json').json()
        for no, label in enumerate(df_comp['source'].unique()):
            colors_dict[label] = colors['custom_colors'][no]
            
    return colors_dict  

In [ ]:
# генерируем цвета из своего списка
colors_dict = colors_for_sources(mode='custom')
colors_dict

Диаграмму будем отрисовывать с помощью Plotly. Для корректной (и более полной) отрисовки нужны следующие данные:  
- sources - список с индексами source  
- targets - список с индексами target  
- values - количество уникальных пользователей, совершивших переход между узлами source-target ("объем" потока между узлами)  
- labels - названия узлов  
- colors_labels - цвет узлов  
- link_color - цвет потоков между узлами  
- link_text - дополнительная информация.  

Следующие 2 функции помогут создать словарь с этими данными: 

In [ ]:
# расчет количества уникальных пользователей в процентах
def percent_users(sources, targets, values):
    
    """
    Расчет уникальных id в процентах (для вывода в hover text каждого узла)
    
    Args:
        sources (list): список с индексами source.
        targets (list): список с индексами target.
        values (list): список с "объемами" потоков.
        
    Returns:
        list: список с "объемами" потоков в процентах
    """
    
    # объединим источники и метки и найдем пары
    zip_lists = list(zip(sources, targets, values))
    
    new_list = []
    
    # подготовим список словарь с общим объемом трафика в узлах
    unique_dict = {}
    
    # проходим по каждому узлу
    for source, target, value in zip_lists:
        if source not in unique_dict:
            # находим все источники и считаем общий трафик
            unique_dict[source] = 0
            for sr, tg, vl in zip_lists:
                if sr == source:
                    unique_dict[source] += vl
                    
    # считаем проценты
    for source, target, value in zip_lists:
        new_list.append(round(100 * value / unique_dict[source], 1))
    
    return new_list

In [ ]:
# создание словаря с данными для отрисовки диаграммы
def lists_for_plot(source_indexes=source_indexes, colors=colors_dict, frac=10):
    
    """
    Создаем необходимые для отрисовки диаграммы переменные списков и возвращаем
    их в виде словаря
    
    Args:
        source_indexes (dict): словарь с именами и индексами source.
        colors (dict): словарь с цветами source.
        frac (int): ограничение на минимальный "объем" между узлами.
        
    Returns:
        dict: словарь со списками, необходимыми для диаграммы.
    """
    
    sources = []
    targets = []
    values = []
    labels = []
    link_color = []
    link_text = []

    # проходим по каждому шагу
    for step in tqdm(sorted(df_comp['step'].unique()), desc='Шаг'):
        if step + 1 not in source_indexes:
            continue

        # получаем индекс источника
        temp_dict_source = source_indexes[step]['sources_dict']

        # получаем индексы цели
        temp_dict_target = source_indexes[step+1]['sources_dict']

        # проходим по каждой возможной паре, считаем количество таких пар
        for source, index_source in tqdm(temp_dict_source.items()):
            for target, index_target in temp_dict_target.items():
                # делаем срез данных и считаем количество id            
                temp_df = df_comp[(df_comp['step'] == step)&(df_comp['source'] == source)&(df_comp['target'] == target)]
                value = len(temp_df)
                # проверяем минимальный объем потока и добавляем нужные данные
                if value > frac:
                    sources.append(index_source)
                    targets.append(index_target)
                    values.append(value)
                    # делаем поток прозрачным для лучшего отображения
                    link_color.append(colors[source].replace(', 1)', ', 0.2)'))
                    
    labels = []
    colors_labels = []
    for key in source_indexes:
        for name in source_indexes[key]['sources']:
            labels.append(name)
            colors_labels.append(colors[name])
            
    # посчитаем проценты всех потоков
    perc_values = percent_users(sources, targets, values)
    
    # добавим значения процентов для howertext
    link_text = []
    for perc in perc_values:
        link_text.append(f"{perc}%")
    
    # возвратим словарь с вложенными списками
    return {'sources': sources, 
            'targets': targets, 
            'values': values, 
            'labels': labels, 
            'colors_labels': colors_labels, 
            'link_color': link_color, 
            'link_text': link_text}

In [ ]:
# создаем словарь
data_for_plot = lists_for_plot()

Приступим к созданию объекта диаграммы.   
Напишем функцию, которая построит диаграмму Санкея, используя подготовленные данные:

In [ ]:
def plot_senkey_diagram(data_dict=data_for_plot):    
    
    """
    Функция для генерации объекта диаграммы Сенкей 
    
    Args:
        data_dict (dict): словарь со списками данных для построения.
        
    Returns:
        plotly.graph_objs._figure.Figure: объект изображения.
    """
    
    fig = go.Figure(data=[go.Sankey(
        domain = dict(
          x =  [0,1],
          y =  [0,1]
        ),
        orientation = "h",
        valueformat = ".0f",
        node = dict(
          pad = 50,
          thickness = 15,
          line = dict(color = "black", width = 0.1),
          label = data_dict['labels'],
          color = data_dict['colors_labels']
        ),
        link = dict(
          source = data_dict['sources'],
          target = data_dict['targets'],
          value = data_dict['values'],
          label = data_dict['link_text'],
          color = data_dict['link_color']
      ))])
    fig.update_layout(title_text="Диаграмма Санкея, визуализация сценариев поведения пользователей", font_size=10, width=1000, height=600)
    
    # возвращаем объект диаграммы
    return fig

In [ ]:
# сохраняем диаграмму в переменную
senkey_diagram = plot_senkey_diagram()

# отобразим полученную диаграмму
senkey_diagram.show()

Исходя из анализа действий пользователей в разрезе сессий, можно сделать следующие выводы:  

К просмотру контактов переходят:  
- 45% пользователей, кликнувших по рекомендованному объявлению  
- 39% пользователей, которые просматривали фотографии в объявлении 
- 18.4% пользователей, которые открывали карту  
- 17.8% пользователей, которые добавили объявление в избранное  
- 17.3% пользователей, которые открывали карточку объявления   
- 15.3% пользователей, которые воспользовались поиском  

Самыми популярными сценариями поведения пользователей, приводящими к целевому действию (просмотр контактов), являются:  

- **Поиск -> просмотр фотографий -> просмотр контактов**     
48.1% пользователей, которые воспользовались поиском, переходят к просмотру фотографий объявления, и 60.7% пользователей, просмотревших фотографии, затем просматривают контакты владельца объявления   
- **Открытие карты -> открытие карточки объявления -> просмотр контактов**    
48.6% пользователей, которые открыли карту, просматривают карточку объявления, и 34.7% пользователей, открывших карточку объявления, переходят к просмотру контактов владельца объявления  
- **Просмотр фотографий -> просмотр контактов -> звонок**   
39% пользователей, которые просматривали фотографии в объявлении, переходят к просмотру контактов владельца объявления, и 73.4% пользователей, просмотревших контакты, совершают звонок через приложение

### Построим воронки по основным сценариям поведения в разрезе уникальных пользователей

#### Сценарий 1:  поиск -> просмотр фотографий -> просмотр контактов:

In [ ]:
# сгруппируем данные по id пользователей, оставив их уникальные действия в системе
grouped_by_id = data.groupby('user_id').agg({'event_name': 'unique'}).reset_index()
grouped_by_id.head()

In [ ]:
# отберем пользователей, совершивших поиск
users_step_1 = grouped_by_id[grouped_by_id['event_name'].apply(lambda x: 'search' in x)]
# отберем пользователей, просмотревших фотографии, среди тех, кто совершал поиск
users_step_2 = users_step_1[users_step_1['event_name'].apply(lambda x: 'photos_show' in x)]
# отберем пользователей, просмотревших контакты, среди тех, кто совершал поиск и просматривал фотографии
users_step_3 = users_step_2[users_step_2['event_name'].apply(lambda x: 'contacts_show' in x)]

In [ ]:
# сохраним в переменную scen_1 количество пользователей на каждом этапе сценария
scen_1 = pd.DataFrame(
            data = {'event_name': ['search', 'photos_show', 'contacts_show'],\
                    'users_amount': [users_step_1['user_id'].nunique(), 
                                     users_step_2['user_id'].nunique(),
                                     users_step_3['user_id'].nunique()]})
scen_1

In [ ]:
# считаем конверсию для первого шага
first_step_conversion = scen_1[:1]['users_amount'].values / data['user_id'].nunique() * 100

# считаем конверсию для 2 и 3 шагов
scen_1['conversion_by_step'] = [*first_step_conversion] + ((scen_1[1:]['users_amount'].values 
                                / scen_1[: len(scen_1) - 1]['users_amount'].values)*100).tolist()
# отобразим результат
scen_1.style.background_gradient(cmap='RdYlGn', subset='conversion_by_step')

In [ ]:
# построим воронку для первого сценария
fig = go.Figure(go.Funnel(y = scen_1['event_name'],
                          x = scen_1['users_amount'],
                          opacity = 0.6,
                          textposition = 'inside',
                          textinfo = 'value + percent previous'))
fig.update_layout(title_text='Воронка событий, сценарий: поиск -> просмотр фотографий -> просмотр контактов')
fig.show()

Конверсия из шага в шаг в первом сценарии поведения пользователей имеет довольно низкие показатели:  
- Около 39% пользователей, воспользовавшихся поиском, переходят к просмотру фотографий  
- Около 29% пользователей, воспользовавшихся поиском и просмотревших фотографии, просматривают контакты владельца объявления 

#### Сценарий 2: открытие карты -> открытие карточки объявления -> просмотр контактов:

In [ ]:
# отберем пользователей, открывших карту
users_step_1 = grouped_by_id[grouped_by_id['event_name'].apply(lambda x: 'map' in x)]
# отберем пользователей, открывших карточку объявления, среди тех, кто открывал карту
users_step_2 = users_step_1[users_step_1['event_name'].apply(lambda x: 'advert_open' in x)]
# отберем пользователей, просмотревших контакты, среди тех, кто открывал карту и карточку объявления
users_step_3 = users_step_2[users_step_2['event_name'].apply(lambda x: 'contacts_show' in x)]

In [ ]:
# сохраним в переменную scen_2 количество пользователей на каждом этапе сценария
scen_2 = pd.DataFrame(
            data = {'event_name': ['map', 'advert_open', 'contacts_show'],\
                    'users_amount': [users_step_1['user_id'].nunique(), 
                                     users_step_2['user_id'].nunique(),
                                     users_step_3['user_id'].nunique()]})
scen_2

In [ ]:
# считаем конверсию для первого шага
first_step_conversion = scen_2[:1]['users_amount'].values / data['user_id'].nunique() * 100

# считаем конверсию для 2 и 3 шагов
scen_2['conversion_by_step'] = [*first_step_conversion] + ((scen_2[1:]['users_amount'].values 
                                / scen_2[: len(scen_2) - 1]['users_amount'].values)*100).tolist()
# отобразим результат
scen_2.style.background_gradient(cmap='RdYlGn', subset='conversion_by_step')

In [ ]:
# построим воронку для второго сценария
fig = go.Figure(go.Funnel(y = scen_2['event_name'],
                          x = scen_2['users_amount'],
                          opacity = 0.6,
                          textposition = 'inside',
                          textinfo = 'value + percent previous'))
fig.update_layout(
    title_text='Воронка событий, сценарий: открытие карты -> открытие карточки объявления -> просмотр контактов')
fig.show()

Показатели конверсии во втором сценарии поведения пользователей в среднем ниже:  
- Около 34% пользователей, открывших карту, открывают карточку объявления    
- Всего 13% пользователей, открывших карту и карточку объявления, просматривают контакты владельца объявления 

#### Сценарий 3: просмотр фотографий -> просмотр контактов -> звонок:

In [ ]:
# отберем пользователей, открывших карту
users_step_1 = grouped_by_id[grouped_by_id['event_name'].apply(lambda x: 'photos_show' in x)]
# отберем пользователей, открывших карточку объявления, среди тех, кто открывал карту
users_step_2 = users_step_1[users_step_1['event_name'].apply(lambda x: 'contacts_show' in x)]
# отберем пользователей, просмотревших контакты, среди тех, кто открывал карту и карточку объявления
users_step_3 = users_step_2[users_step_2['event_name'].apply(lambda x: 'contacts_call' in x)]

In [ ]:
# сохраним в переменную scen_3 количество пользователей на каждом этапе сценария
scen_3 = pd.DataFrame(
            data = {'event_name': ['photos_show', 'contacts_show', 'contacts_call'],\
                    'users_amount': [users_step_1['user_id'].nunique(), 
                                     users_step_2['user_id'].nunique(),
                                     users_step_3['user_id'].nunique()]})
scen_3

In [ ]:
# считаем конверсию для первого шага
first_step_conversion = scen_3[:1]['users_amount'].values / data['user_id'].nunique() * 100

# считаем конверсию для 2 и 3 шагов
scen_3['conversion_by_step'] = [*first_step_conversion] + ((scen_3[1:]['users_amount'].values 
                                / scen_3[: len(scen_3) - 1]['users_amount'].values)*100).tolist()
# отобразим результат
scen_3.style.background_gradient(cmap='RdYlGn', subset='conversion_by_step')

In [ ]:
# построим воронку для третьего сценария
fig = go.Figure(go.Funnel(y = scen_3['event_name'],
                          x = scen_3['users_amount'],
                          opacity = 0.6,
                          textposition = 'inside',
                          textinfo = 'value + percent previous'))
fig.update_layout(
    title_text='Воронка событий, сценарий: просмотр фотографий -> просмотр контактов -> звонок')
fig.show()

Показатели конверсии третьего сценария поведения пользователей являются средними:
- Около 30% пользователей, просмотревших фотографии, просматривают контакты владельца объявления
- Около 46% пользователей, просмотревших фотографии и контакты владельца объявления, совершают звонок через приложение  

### Проверим, как различается время между событиями map -> contacts_show и search -> contacts_show в рамках сессий

Для этого:  
* найдем сессии, в которых встречаются события map и show_contacts  
* рассчитаем время от map до show_contacts  
* выведем среднее и медиану данного показателя  

И такие же действия проделаем для пары search -> contacts_show.

#### Пара map -> contacts_show:

In [ ]:
# сгруппируем данные по сессиям
gr_by_sessions = data.groupby('session_id')\
               .agg({'event_name':'unique'})\
               .reset_index()
gr_by_sessions.head()

In [ ]:
# найдем сессии, в которых встречаются действия map и show_contacts
map_contacts = gr_by_sessions[gr_by_sessions['event_name'].apply(lambda x: 'map' in x and 'contacts_show' in x)]
map_contacts.head()

In [ ]:
# сохраним id этих сессий в отдельный список
s_id = map_contacts['session_id'].unique().tolist()
s_id[:5]

In [ ]:
# оставим только необходимые данные(номер сессии, действие и время действия) 
slice = data[['session_id', 'event_name', 'event_time']]

# оставим только такие id сессий, в которых встречаются действия map и show_contacts
slice = slice[slice['session_id'].isin(s_id)]

# удалим дубликаты действий в полученном срезе
slice = slice.drop_duplicates(subset=['session_id', 'event_name'])
slice.head()

In [ ]:
# оставим только действия map и contacts_show, чтобы рассчитать время между ними
slice = slice[(slice['event_name'] == 'contacts_show') | (slice['event_name'] == 'map')]
slice.head()

In [ ]:
# исключим сессии, в которых действие contacts_show раньше map
s = slice.groupby('session_id', as_index=False).agg({'event_name':list})
s = s[s['event_name'].apply(lambda x: x[1] == 'contacts_show')]

In [ ]:
# сохраним id оставшихся сессий в отдельный список
s_id = s['session_id'].unique().tolist()

# отфильтруем срез по оставшимся id сессий
slice = slice[slice['session_id'].isin(s_id)]
slice.head()

In [ ]:
# рассчитаем время между действиями map и contacts_show
slice['time_diff'] = slice.groupby('session_id')['event_time'].diff().abs()
slice.head()

In [ ]:
# посмотрим на характерные значения стоблца со временем между событиями
slice['time_diff'].describe()

Среднее время в рамках сессий между событиями map и contacts_show составляет 8 минут 39 секунд, медианное время - 4 минуты 56 секунд.

#### Пара search -> contacts_show:

In [ ]:
# найдем сессии, в которых встречаются действия search и contacts_show
search_contacts = gr_by_sessions[gr_by_sessions['event_name'].apply(lambda x: 'search' in x and 'contacts_show' in x)]
search_contacts.head()

In [ ]:
# сохраним id этих сессий в отдельный список
s_id2 = search_contacts['session_id'].unique().tolist()

In [ ]:
# оставим только необходимые данные(номер сессии, действие и время действия) 
slice = data[['session_id', 'event_name', 'event_time']]

# оставим только такие id сессий, в которых встречаются действия search и show_contacts
slice = slice[slice['session_id'].isin(s_id2)]

# удалим дубликаты действий в полученном срезе
slice = slice.drop_duplicates(subset=['session_id', 'event_name'])

# оставим только действия search и contacts_show, чтобы рассчитать время между ними
slice = slice[(slice['event_name'] == 'search') | (slice['event_name'] == 'contacts_show')]

# исключим сессии, в которых действие contacts_show раньше search
s = slice.groupby('session_id', as_index=False).agg({'event_name':list})
s = s[s['event_name'].apply(lambda x: x[1] == 'contacts_show')]

# сохраним id оставшихся сессий в отдельный список
s_id2 = s['session_id'].unique().tolist()

# отфильтруем срез по оставшимся id сессий
slice = slice[slice['session_id'].isin(s_id2)]

# рассчитаем время между действиями search и contacts_show
slice['time_diff'] = slice.groupby('session_id')['event_time'].diff().abs()
slice.head()

In [ ]:
# посмотрим на характерные значения столбца со временем между событиями
slice['time_diff'].describe()

Среднее время в рамках сессий между событиями search и contacts_show составляет 5 минут 43 секунды, медианное время - 3 минуты 42 секунды.  

При сравнении показателей времени стоит опираться на медианные значения, чтобы исключить влияние выбросов и аномальных значений.  
- Медианное время между событиями map -> contacts_show: 4 минуты 56 секунд  
- Медианное время между событиями search -> contacts_show: 3 минуты 42 секунды  

Сравнивания медианное время данных пар действий, можно сделать вывод о том, что цепочка событий search -> contacts_show проходит быстрее на 1 минуту 14 секунд.

### Оценим, какие действия чаще совершают пользователи, которые просматривают контакты

Для того, чтобы выявить наиболее частые действия среди пользователей, которые просматривают контакты, сделаем следующее:
- разделим пользователей на 2 группы : просматривающих контакты и не просматривающих контакты  
- посчитаем для каждой из групп количество действий пользователей 
- проанализируем и сравним показатели в каждой из групп

#### Группа пользователей, просматривающих контакты:

In [ ]:
# отберем id пользователей, которые совершали действие contacts_show
contact_users = data[data['event_name'] == 'contacts_show']['user_id'].unique().tolist()

In [ ]:
# отберем данные только по пользователям, совершившим действие contacts_show и посчитаем количество их действий
act_contact_users = data.query('user_id in @contact_users')\
                        .groupby('event_name', as_index=False)\
                        ['user_id'].count()\
                        .sort_values('user_id', ascending=False)\
                        .rename(columns={'user_id': 'amount'}) 

# сохраним общее количество действий среди пользователей c contacts_show
all_act = data.query('user_id in @contact_users')['event_name'].count()

act_contact_users['share_of_all_actions'] = round(act_contact_users['amount'] / all_act * 100, 1)

act_contact_users.style.background_gradient(cmap='RdYlGn', subset='share_of_all_actions')

Самыми частым действием среди пользователей, просматривающих контакты, является просмотр рекомендованных объявлений (занимает 41.6% от всех действий). На втором месте идет само целевое действие - просмотр контактов (18.6% от всех действий), далее следует просмотр фотографий (16% от всех действий).  

Реже всего в данной группе пользователи кликают по рекомендованным объявлениям (1.3% от всех действий), добавляют объявления в избранное (2.1% от всех действий) и звонят через приложение (2.9% от всех действий).

#### Группа пользователей, не просматривающих контакты:

In [ ]:
# отберем данные по пользователям без действия contacts_show и посчитаем количество их действий
act_not_contact_users = data.query('user_id not in @contact_users')\
                        .groupby('event_name', as_index=False)\
                        ['user_id'].count()\
                        .sort_values('user_id', ascending=False)\
                        .rename(columns={'user_id': 'amount'}) 

# сохраним общее количество действий среди пользователей c contacts_show
all_act = data.query('user_id not in @contact_users')['event_name'].count()

act_not_contact_users['share_of_all_actions'] = round(act_not_contact_users['amount'] / all_act * 100, 1)

act_not_contact_users.style.background_gradient(cmap='RdYlGn', subset='share_of_all_actions')

Самыми частым действием среди пользователей, не просматривающих контакты, также является просмотр рекомендованных объявлений (занимает 58% от всех действий). На втором месте идет просмотр фотографий (13.9% от всех действий), на третьем - поиск в системе (10.7% от всех действий).  

Реже всего в данной группе пользователи добавляют объявления в избранное (1.8% от всех действий) и кликают по рекомендованным объявлениям (1.1% от всех действий).

Анализируя обе группы, можно сделать вывод, что пользователи, просматривающие контакты:  
- на 16.4% реже видят рекомендованные объявления  
- на 4.2% реже открывают карточки объявлений 
- на 2.3% реже просматривают карту  
- на 1.2% реже пользуются поиском 

- на 4.7% чаще просматривают фотографии  
- на 0.3% чаще добавляют объявления в избранное  
- на 0.2% чаще кликают по рекомендованным объявлениям  
- звонят через приложение, в отличие от группы, которая не просматривает контакты 

## Проверка гипотез

### Гипотеза 1  
**H0**: Конверсия в просмотры контактов у пользователей, совершивших действие tips_show (увидел рекомендованное объявление) и у пользователей, совершивших действие tips_click (кликнул по рекомендованному объявлению), равна  

**H1**: Конверсия в просмотры контактов у пользователей, совершивших действие tips_show (увидел рекомендованное объявление)  и у пользователей, совершивших действие tips_click (кликнул по рекомендованному объявлению), не равна 

Подготовим данные для проверки:  
- пользователей, совершивших действие tips_show (без tips_click), определим как группу А  
- пользователей, совершивших действие tips_click и tips_click, определим как группу В

In [ ]:
# отберем id пользователей, совершивших действие tips_show
tips_show_users = data[data['event_name'] == 'tips_show']['user_id'].unique().tolist()

# отберем id пользователей, совершивших действие tips_click
tips_click_users = data[data['event_name'] == 'tips_click']['user_id'].unique().tolist()

In [ ]:
# сохраним данные по группе А (tips_show без tips_click)
a = data.query('user_id in @tips_show_users and user_id not in @tips_click_users')
# сохраним данные по группе В (tips_show + tips_click)
b = data.query('user_id in @tips_click_users and user_id in @tips_show_users')

Для проверки гипотезы о равенстве пропорций двух генеральных совокупностей проведем z-тест:

In [ ]:
# критический уровень статистической значимости
alpha = .05
 
# число пользователей в группе 1 и группе 2:
n_users = np.array([a['user_id'].nunique(), 
                    b['user_id'].nunique()])

# число пользователей, совершивших событие в группе 1 и группе 2
success = np.array([a[a['event_name'] == 'contacts_show']['user_id'].nunique(), 
                    b[b['event_name'] == 'contacts_show']['user_id'].nunique()])

print(success, n_users)

# пропорции успехов в группах:
p1 = success[0]/n_users[0]
p2 = success[1]/n_users[1]
    
# пропорция успехов в комбинированном датасете:
p_combined = (success[0] + success[1]) / (n_users[0] + n_users[1])

# разница пропорций в датасетах
difference = p1 - p2 

# считаем статистику в ст.отклонениях стандартного нормального распределения
z_value = difference /  np.sqrt(p_combined * (1 - p_combined) * (1/n_users[0] + 1/n_users[1]))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = stats.norm(0, 1)  

p_value = (1 - distr.cdf(abs(z_value))) * 2   #тест двусторонний, удваиваем результат
    
print('Событие: contacts_show')
print('p-значение: ', p_value)

if p_value < alpha:
    print('Отвергаем нулевую гипотезу: между долями есть разница')
else:
    print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')

A/B тестирование двух групп пользователей выявило достаточную разницу в конверсиях, чтобы говорить о статистически значимом различии между группами. Это означает, нулевая гипотеза о равенстве конверсии в просмотры контактов у пользователей, совершивших действие tips_show и у пользователей, совершивших действие tips_click, отвергнута. 

### Гипотеза 2   
**H0**: Длительности сессий пользователей, совершивших целевое действие contacts_show и тех, кто его не совершал, равны  
**H1**: Длительности сессий у пользователей, совершивших целевое действие contacts_show и тех, кто его не совершал, не равны  

Посчитаем длительность сессий пользователей и сохраним значения в переменной *user_sessions:*

In [ ]:
# сгруппируем данные по id пользователя и id сессии и отобразим время совершения каждого действия
user_sessions = data.groupby(['user_id', 'session_id'], as_index=False).agg({'timestamp':list})
user_sessions.head()

In [ ]:
# вычтем из времени последнего действия в разрезе сессии время первого действия и запишем в столбец time_spent
user_sessions['time_spent'] = user_sessions['timestamp'].apply(lambda x: x[-1] - x[0])
user_sessions.head()

Ранее мы уже сохраняли id пользователей, совершивших действие contacts_show, в переменную contact_users.  
Разделим с помощью нее пользователей на 2 группы:  
- пользователей, совершивших действие contacts_show, определим как группу А  
- пользователей, не совершавших действие contacts_show, определим как группу В

In [ ]:
a = user_sessions.query('user_id in @contact_users')
b = user_sessions.query('user_id not in @contact_users')

In [ ]:
# сохраним время сессий группы А, сгруппировав данные по id пользователей
time_users_a = a.groupby('user_id', as_index=False).agg({'time_spent': pd.Series.nunique})
# сохраним время сессий группы В, сгруппировав данные по id пользователей
time_users_b = b.groupby('user_id', as_index=False).agg({'time_spent': pd.Series.nunique})

Проверим тестируемые данные на нормальность распределения с помощью критерия Шапиро-Уилка *stats.shapiro().*   

Сформулируем гипотезы для проверки распределения:  
**H0**: данные о длительности сессиий имеют нормальное распределение   
**H1**: данные о длительности сессиий распределены не нормально   

In [ ]:
# критический уровень статистической значимости
alpha = 0.05  

# проверка распределения в группе А
results = stats.shapiro(time_users_a['time_spent'])

print('p-значение: ', results[1]) # второе значение в массиве результатов (с индексом 1) - p-value

if results[1] < alpha:
    print('Отвергаем нулевую гипотезу: распределение не нормально')
else:
    print('Не получилось отвергнуть нулевую гипотезу, всё нормально')

In [ ]:
# проверка распределения в группе В
results = stats.shapiro(time_users_b['time_spent'])

print('p-значение: ', results[1])

if results[1] < alpha:
    print('Отвергаем нулевую гипотезу: распределение не нормально')
else:
    print('Не получилось отвергнуть нулевую гипотезу, всё нормально')

По результатам тестов распределение не является нормальным ни в одной из тестируемых групп.
Это означает, что в данных есть большие (по сравнению с нормальным распределением) выбросы. В таком случае, для дальнейшего расчёта статистической значимости прибегнем к непараметрическому тесту Уилкоксона-Манна-Уитни.

In [ ]:
# рассчитаем p-value c помощью теста Уилкоксона-Манна-Уитни
print('P-value: ', stats.mannwhitneyu(time_users_a['time_spent'], 
                   time_users_b['time_spent'], 
                   alternative='two-sided')[1])

# рассчитаем различия в длительности сессий между группам
print('Относительные различия в длительности сессий между группами: ''{0:.3f}'.format(
    time_users_a['time_spent'].mean()
    /time_users_b['time_spent'].mean()-1))

P-value значительно больше 0.05. Несмотря на большой показатель различия между группами (46.5%), причин отвергать нулевую гипотезу и считать, что в длительности сессий пользователей, совершивших целевое действие contacts_show и пользователей, которые его не совершали, есть различия, нет.

## Выводы и рекомендации по результатам исследования

**Общие выводы по исследованию:**  
- Период исследования, за который проводился анализ, составляет 27 дней  
- За это время приложенияем воспользовались 4239 человек  
- Больше всего пользователей приходит через Яндекс (привлечено 45% от общего числа уникальных пользователей), а меньше всего через Google - 26.3% пользователей  
- Самым частым действием в приложении, инициированным пользователем, является поиск - им воспользовались 38.8% пользователей  
- Целевое действие - просмотр контактов, совершают около 23% пользователей  
- Популярными сценариями поведения пользователей, приводящими к целевому действию (просмотр контактов), являются следующие последовательности:   
1. Поиск -> просмотр фотографий -> просмотр контактов   
2. Открытие карты -> открытие карточки объявления -> просмотр контактов   
3. Просмотр фотографий -> просмотр контактов -> звонок  
- Самым частым действием, инициированным пользователем, среди пользователей, просматривающих контакты, является просмотр фотографий (16% от всех действий) 
- Реже всего пользователи, просматривающие контакты, кликают по рекомендованным объявлениям (1.3% от всех действий), добавляют объявления в избранное (2.1% от всех действий) и звонят через приложение (2.9% от всех действий)   
- Цепочка действий поиск -> просмотр контактов проходит в среднем быстрее на 1 минуту 14 секунд по сравнению с цепочкой действий открытие карты -> просмотр контактов  



В процессе исследования проверены две пары статистических гипотез:

- Гипотеза о равенстве конверсий в просмотры контактов у пользователей, совершивших действие tips_show и у пользователей, совершивших действие tips_click, отвергнута. Следовательно, можно сделать вывод, что между долями в конверсии у пользователей, увидевших рекомендованные объявления, и пользователей, кликнувших по рекомендованному объявлению, есть статистически значимая разница.  

- Гипотеза о равенстве длительности сессий пользователей, совершивших целевое действие contacts_show и тех, кто его не совершал, подтверждена. Следовательно, статистически значимой разницы в длительности сессий между пользователями, которые просматривали контакты и теми, кто этого не делал, нет.  


**Рекомендации для привлечения и увеличения вовлеченности пользователей:**       

- Обратить внимание на канал привлечения Google, так как он является одной из популярных поисковых систем, и может существенно увеличить приток новых пользователей в приложение  
- Усовершенствовать процесс звонков через приложение. Сейчас этим инструментом пользуется всего 5% пользователей, что может свидетельствовать о том, что пользователям неудобно использовать  
- Улучшить алгоритмы подбора рекомендованных объявлений. Cейчас всего 7.5% пользователей кликают по рекомендованным объявлением, что может говорить о том, что алгоритмы приложения предлагают пользователям нерелевантые объявления  
- Произвести сегментацию пользователей на группы, в зависимости от их сценариев поведения - это позволит более точечно влиять их вовлечение и улучшать показатели конверсии    